# Temperature Analysis

In [ ]:
import pandas
import numpy

from bokeh.plotting import output_notebook, show, figure, ColumnDataSource
from bokeh.models import Span
from bokeh.io import push_notebook
output_notebook()

from ipywidgets import widgets, interactive
from IPython.display import display

# Get current sensors database from raspberry pi
df = pandas.read_csv('sensors.log',names = ["Level", "Sensor", "Time", "Value"],
                    parse_dates=['Time'])

df.index.name = 'ID'

# Get temperature values
temp = df.loc[df['Sensor'] == 'T']

# Draw temperature vs time
p = figure(width=850, height=350, x_axis_type="datetime")
p.line(temp['Time'],temp['Value'],line_width=2)
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Temperature'

#Init threshold lines
minline = Span(dimension='width', line_color='blue', line_width=1)
maxline = Span(dimension='width', line_color='red', line_width=1)
p.add_layout(minline)
p.add_layout(maxline)

# Callback to get user limits
def Analysis(Tmin, Tmax):
    print('Safe range of temperatures (°C): [{}, {}] '.format(Tmin, Tmax))
    minline.location = Tmin
    maxline.location = Tmax
    push_notebook()

# Display description
w = interactive(Analysis, Tmin=(-40.0,70.0), Tmax=(-40.0,70.0),__manual=True)
display(w)

In [ ]:
show(p, notebook_handle=True)

In [ ]:
df = pandas.read_csv('sensors.log',names = ["Level", "Sensor", "Time", "Value"],
                    parse_dates=['Time'])
temp = df.loc[df['Sensor'] == 'T']

In [ ]:
temp.reset_index(inplace=True)
temp_id = temp [['Time','Value']]

In [ ]:
temp_sel = temp_id[(temp_id['Value'] < 15) | (temp_id['Value'] > 40.5)].copy()
temp_sel

In [ ]:
temp_group = temp_sel.groupby(temp_sel.index - numpy.arange(temp_sel.shape[0]))
for _, group in temp_group:
#    print(i[1].iloc[[0, -1]])
    print(group)

TODO
index / start date / stop date / duration / max / average ?

select plot interval by index